In [1]:
#this script

import numpy as np
import pandas as pd
import ast
import tmdbsimple as tmdb
import pickle

In [2]:
#read in the credits file 
credits_data = pd.read_csv("./tmdb-5000-movie-dataset/tmdb_5000_credits.csv")

print(credits_data.columns)



Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')


In [30]:
# director_info = np.array([]) #numpy array to store director IMDB ID, name and popularity
director_info = []
columns = ['id', 'name', 'popularity']
#iterate over credits_data row by row and add director info to director_info array
crew = credits_data['crew']

#each row in crew is an array with dicts
#look for "department": "Directing" or "job": "Director"

for row in crew:
    row = ast.literal_eval(row)
    for credit_dict in row:
        if credit_dict['job'] == 'Director':
            temp = [credit_dict['id'], credit_dict['name']]
            director_info.append(temp)
            
print(len(director_info))


5166


In [31]:
print(director_info[:10])
director_info_dict = {} #key:id and value = {name:"", popularity:""}

[[2710, 'James Cameron'], [1704, 'Gore Verbinski'], [39, 'Sam Mendes'], [525, 'Christopher Nolan'], [7, 'Andrew Stanton'], [7623, 'Sam Raimi'], [76595, 'Byron Howard'], [121830, 'Nathan Greno'], [12891, 'Joss Whedon'], [11343, 'David Yates']]


In [32]:
#iterate over director_info array and add a new element to each row. New element is popularity for director
tmdb.API_KEY = 'set your own'

for row in director_info:
    director_id = row[0]
    director = tmdb.People(director_id)
    #make call
    response = director.info()
    row.append(director.popularity)
    #also save to director_info_dict
    director_info_dict[director_id] = {'name': director.name,'popularity': director.popularity}

In [33]:
print(director_info[:10])
print(director_info_dict)

[[2710, 'James Cameron', 4.973699], [1704, 'Gore Verbinski', 3.107182], [39, 'Sam Mendes', 2.667], [525, 'Christopher Nolan', 3.160919], [7, 'Andrew Stanton', 1.588633], [7623, 'Sam Raimi', 1.930133], [76595, 'Byron Howard', 1.01058], [121830, 'Nathan Greno', 2.9e-05], [12891, 'Joss Whedon', 2.824829], [11343, 'David Yates', 2.936039]]
{2710: {'name': 'James Cameron', 'popularity': 4.973699}, 1704: {'name': 'Gore Verbinski', 'popularity': 3.107182}, 39: {'name': 'Sam Mendes', 'popularity': 2.667}, 525: {'name': 'Christopher Nolan', 'popularity': 3.160919}, 7: {'name': 'Andrew Stanton', 'popularity': 1.588633}, 7623: {'name': 'Sam Raimi', 'popularity': 1.930133}, 76595: {'name': 'Byron Howard', 'popularity': 1.01058}, 121830: {'name': 'Nathan Greno', 'popularity': 2.9e-05}, 12891: {'name': 'Joss Whedon', 'popularity': 2.824829}, 11343: {'name': 'David Yates', 'popularity': 2.936039}, 15217: {'name': 'Zack Snyder', 'popularity': 4.108712}, 9032: {'name': 'Bryan Singer', 'popularity': 3.5

In [34]:
#save this information in a pandas dataframe
df_director_info = pd.DataFrame(data = director_info, columns= ['id', 'name', 'popularity'])

In [35]:
df_director_info.head()

,id,name,popularity
0,2710,James Cameron,4.973699
1,1704,Gore Verbinski,3.107182
2,39,Sam Mendes,2.667000
3,525,Christopher Nolan,3.160919
4,7,Andrew Stanton,1.588633


In [36]:
#save the dataframe to csv
df_director_info.to_csv('director_with_popularity_21Nov.csv', sep=',')


In [39]:
#pickle the director_info_dict
with open('director_info_dict_pkl','wb') as f:
    pickle.dump(director_info_dict, f)

In [45]:
#see if properly pickled
with open('director_info_dict_pkl','rb') as f:
    director_dict_loaded = pickle.load(f)

print(director_dict_loaded[2710]['popularity'] == 4.973699)

True


In [3]:
#for each movie get popularity of first 5 cast members
cast_info = {}
columns = ['id', 'name', 'popularity']
#iterate over credits_data row by row and add cast info to cast_info array
cast = credits_data['cast']

for row in cast:
    row = ast.literal_eval(row)
    #consider only first 5 cast members
    row = row[:3]
    for cast_dict in row:
        if cast_dict["id"] not in cast_info:
            cast_info[cast_dict["id"]] = {'name': cast_dict["name"]}

In [4]:
cast_info

{65731: {'name': 'Sam Worthington'},
 8691: {'name': 'Zoe Saldana'},
 10205: {'name': 'Sigourney Weaver'},
 85: {'name': 'Johnny Depp'},
 114: {'name': 'Orlando Bloom'},
 116: {'name': 'Keira Knightley'},
 8784: {'name': 'Daniel Craig'},
 27319: {'name': 'Christoph Waltz'},
 121529: {'name': 'Léa Seydoux'},
 3894: {'name': 'Christian Bale'},
 3895: {'name': 'Michael Caine'},
 64: {'name': 'Gary Oldman'},
 60900: {'name': 'Taylor Kitsch'},
 21044: {'name': 'Lynn Collins'},
 2206: {'name': 'Samantha Morton'},
 2219: {'name': 'Tobey Maguire'},
 205: {'name': 'Kirsten Dunst'},
 17051: {'name': 'James Franco'},
 69899: {'name': 'Zachary Levi'},
 16855: {'name': 'Mandy Moore'},
 2517: {'name': 'Donna Murphy'},
 3223: {'name': 'Robert Downey Jr.'},
 74568: {'name': 'Chris Hemsworth'},
 103: {'name': 'Mark Ruffalo'},
 10980: {'name': 'Daniel Radcliffe'},
 10989: {'name': 'Rupert Grint'},
 10990: {'name': 'Emma Watson'},
 880: {'name': 'Ben Affleck'},
 73968: {'name': 'Henry Cavill'},
 90633: {

In [6]:
#get popularity metric for each cast member
for id in cast_info:
    cast_member = tmdb.People(id)
    #make call
    response = cast_member.info()
    cast_info[id]['popularity'] = cast_member.popularity
    
#popularity could be a very small number    

#pickle the cast_info_dict
with open('cast_info_dict_pkl','wb') as f:
    pickle.dump(cast_info, f)

In [20]:
# #pickle the cast_info_dict
# with open('cast_info_dict_pkl','wb') as f:
#     pickle.dump(cast_info, f)

In [9]:
# np.save('cast_info.npy', cast_info)
#save it to file 
#see if properly pickled
with open('cast_info_dict_pkl','rb') as f:
    new_load = pickle.load(f)
print(new_load)

{65731: {'name': 'Sam Worthington', 'popularity': 9.506056}, 8691: {'name': 'Zoe Saldana', 'popularity': 7.86875}, 10205: {'name': 'Sigourney Weaver', 'popularity': 3.557153}, 85: {'name': 'Johnny Depp', 'popularity': 15.493812}, 114: {'name': 'Orlando Bloom', 'popularity': 5.16579}, 116: {'name': 'Keira Knightley', 'popularity': 8.094822}, 8784: {'name': 'Daniel Craig', 'popularity': 9.294077}, 27319: {'name': 'Christoph Waltz', 'popularity': 9.395564}, 121529: {'name': 'Léa Seydoux', 'popularity': 4.897905}, 3894: {'name': 'Christian Bale', 'popularity': 7.330192}, 3895: {'name': 'Michael Caine', 'popularity': 5.084451}, 64: {'name': 'Gary Oldman', 'popularity': 6.203581}, 60900: {'name': 'Taylor Kitsch', 'popularity': 5.091925}, 21044: {'name': 'Lynn Collins', 'popularity': 0.803125}, 2206: {'name': 'Samantha Morton', 'popularity': 2.019743}, 2219: {'name': 'Tobey Maguire', 'popularity': 2.330537}, 205: {'name': 'Kirsten Dunst', 'popularity': 4.009943}, 17051: {'name': 'James Franco

In [ ]:
#there is a bug somewhere in the script above and basically all popularity are the same because of this bug